### Import Datasets

In [1]:
import pandas as pd

# Read dataframes
df_actions = pd.read_excel("data/actions.xlsx")
df_clients = pd.read_excel("data/clients.xlsx")
df_transactions = pd.read_excel("data/transactions.xlsx")

/Users/ghalichraibi/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Visualize dataframes
df_actions.head()

,action_id,action_type_label,action_subcategory_label,action_start_date,action_year,action_end_date,action_collection_year,action_collection,action_universe,action_category_label,action_channel,action_label,client_id,client_is_present,client_is_invited
0,a654760,Collection,Collection,2022-04-07,2022,2022-04-11,2022,Mid-Season Jewelry and Timepieces,Timepieces and Jewelry,Retail,In store,Exclusive Offer,c23083988,1,1
1,a979979,Collection,Collection,2021-10-20,2021,2021-10-21,2021,Sailor Collection,Women's Fashion,Retail,In store,Exclusive Offer,c40865409,1,1
2,a803469,Collection,Collection,2020-10-29,2020,2020-10-29,2020,Spring Collection,Men's Fashion,Retail,In store,Social Gathering,c95435465,1,1
3,a887515,Collection,Collection,2022-01-11,2022,2022-01-11,2022,Formal Retail Action,Men's Fashion,Retail,In store,Social Gathering,c58792918,1,1
4,a233620,Collection,Collection,2021-05-27,2021,2021-06-21,2021,Mid-Season Jewelry and Timepieces,Timepieces and Jewelry,Retail,NaN,Exclusive Offer,c34394230,1,1


In [3]:
df_clients.head()

,client_id,client_country,client_gender,client_nationality,client_city,client_premium_status,client_is_phone_contactable,client_is_email_contactable,client_is_instant_messaging_contactable,client_is_contactable
0,c41404743,BN,NaN,NaN,NaN,0,0,0,1,1
1,c65630254,CA,NaN,NaN,NaN,0,0,0,1,1
2,c89796498,NL,NaN,NaN,NaN,0,0,0,1,1
3,c03959902,SE,NaN,NaN,NaN,0,0,0,1,1
4,c39809280,BG,NaN,NaN,NaN,0,0,0,1,1


In [4]:
df_transactions.head()

,client_id,transaction_id,transaction_date,product_quantity,gross_amount_euro,product_category,product_subcategory,product_style
0,c17974679,t11288118,2020-07-25,1,3044,Women Ready-to-Wear,Unknown,Casual Style
1,c17974679,t93389142,2020-06-20,1,569,Women Accessory,Unknown,Stylish Fashion
2,c17974679,t62924895,2021-01-13,1,7727,Women Bags,Unknown,Leather
3,c17974679,t93389142,2020-06-20,1,3954,Women Ready-to-Wear,Unknown,Casual Style
4,c17974679,t33368160,2021-05-03,1,1541,Women Ready-to-Wear,bottom segment,Casual Style


###  Compute key KPIs at action_label granularity

In [5]:
# Convert date columns to datetime
df_actions["action_start_date"] = pd.to_datetime(df_actions["action_start_date"])
df_actions["action_end_date"] = pd.to_datetime(df_actions["action_end_date"])

# Compute average action duration per action_label
df_actions["action_duration"] = (df_actions["action_end_date"] - df_actions["action_start_date"]).dt.days
df_actions.head()

,action_id,action_type_label,action_subcategory_label,action_start_date,action_year,action_end_date,action_collection_year,action_collection,action_universe,action_category_label,action_channel,action_label,client_id,client_is_present,client_is_invited,action_duration
0,a654760,Collection,Collection,2022-04-07,2022,2022-04-11,2022,Mid-Season Jewelry and Timepieces,Timepieces and Jewelry,Retail,In store,Exclusive Offer,c23083988,1,1,4
1,a979979,Collection,Collection,2021-10-20,2021,2021-10-21,2021,Sailor Collection,Women's Fashion,Retail,In store,Exclusive Offer,c40865409,1,1,1
2,a803469,Collection,Collection,2020-10-29,2020,2020-10-29,2020,Spring Collection,Men's Fashion,Retail,In store,Social Gathering,c95435465,1,1,0
3,a887515,Collection,Collection,2022-01-11,2022,2022-01-11,2022,Formal Retail Action,Men's Fashion,Retail,In store,Social Gathering,c58792918,1,1,0
4,a233620,Collection,Collection,2021-05-27,2021,2021-06-21,2021,Mid-Season Jewelry and Timepieces,Timepieces and Jewelry,Retail,NaN,Exclusive Offer,c34394230,1,1,25


In [6]:
# Compute average action time per action label without duplicates
unique_actions = df_actions.drop_duplicates(subset=["action_id"])

# Compute average action duration per action label
avg_duration = unique_actions.groupby("action_label")["action_duration"].mean()

# Number of distinct events (unique action_id count per action label)
num_events = df_actions.groupby("action_label")["action_id"].nunique()

# Average unique clients invited per action label
avg_clients_invited = df_actions.groupby("action_label").apply(
    lambda x: x.groupby("action_id")["client_id"].nunique().mean()
)

# Average unique clients present per action label
avg_clients_present = df_actions[df_actions["client_is_present"] == 1].groupby("action_label").apply(
    lambda x: x.groupby("action_id")["client_id"].nunique().mean()
)

# Conversion rate per action label (avg unique clients present / avg unique clients invited)
conversion_rate = avg_clients_present / avg_clients_invited

# Combine all metrics into a single DataFrame
summary_df = pd.DataFrame({
    "num_distinct_events": num_events,
    "avg_duration": avg_duration,
    "avg_unique_clients_invited": avg_clients_invited,
    "avg_unique_clients_present": avg_clients_present,
    "conversion_rate": conversion_rate
}).reset_index()

# Sort by conversion rate in descending order
summary_df.sort_values(by="num_distinct_events", ascending=False)

/var/folders/tv/2gb1mgj54p13rlxnb7v41s4w0000gn/T/ipykernel_42324/1895587326.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_clients_invited = df_actions.groupby("action_label").apply(
/var/folders/tv/2gb1mgj54p13rlxnb7v41s4w0000gn/T/ipykernel_42324/1895587326.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_clients_present = df_actions[df_actions["client_is_present"] == 1].groupby("action_l

,action_label,num_distinct_events,avg_duration,avg_unique_clients_invited,avg_unique_clients_present,conversion_rate
3,Exclusive Offer,325,4.683077,7.603077,5.318008,0.699455
13,Social Gathering,67,3.059701,54.134328,13.790323,0.254743
1,Business Engagement,36,6.694444,2.583333,2.583333,1.000000
15,Trunk Show Event,35,2.514286,13.085714,13.028571,0.995633
6,Holiday Celebration,27,2.222222,89.037037,92.307692,1.036734
11,Miscellaneous Social Gathering,24,4.833333,6.250000,6.476190,1.036190
12,Outdoor Event,19,4.052632,6.631579,2.466667,0.371958
10,Luxurious Fur Showcase,10,1.900000,1.200000,1.222222,1.018519
9,Lunar New Year Celebration,9,1.333333,67.111111,4.000000,0.059603
2,Exclusive Client Days,4,7.000000,3.500000,1.750000,0.500000


### Determine first & last transaction after a specific action to find which ones are the most effective

In [7]:
# Merge datasets
df_merged = df_transactions.merge(df_actions, on="client_id", how="left")
df_merged["total_revenue"] = df_merged["product_quantity"] * df_merged["gross_amount_euro"]
 
df_merged.head()

,client_id,transaction_id,transaction_date,product_quantity,gross_amount_euro,product_category,product_subcategory,product_style,action_id,action_type_label,...,action_collection_year,action_collection,action_universe,action_category_label,action_channel,action_label,client_is_present,client_is_invited,action_duration,total_revenue
0,c17974679,t11288118,2020-07-25,1,3044,Women Ready-to-Wear,Unknown,Casual Style,a066443,Business Treatment,...,2020.0,Winter Collection,Women's Fashion,Client,Outside venue,Business Engagement,1.0,1.0,0.0,3044
1,c17974679,t93389142,2020-06-20,1,569,Women Accessory,Unknown,Stylish Fashion,a066443,Business Treatment,...,2020.0,Winter Collection,Women's Fashion,Client,Outside venue,Business Engagement,1.0,1.0,0.0,569
2,c17974679,t62924895,2021-01-13,1,7727,Women Bags,Unknown,Leather,a066443,Business Treatment,...,2020.0,Winter Collection,Women's Fashion,Client,Outside venue,Business Engagement,1.0,1.0,0.0,7727
3,c17974679,t93389142,2020-06-20,1,3954,Women Ready-to-Wear,Unknown,Casual Style,a066443,Business Treatment,...,2020.0,Winter Collection,Women's Fashion,Client,Outside venue,Business Engagement,1.0,1.0,0.0,3954
4,c17974679,t33368160,2021-05-03,1,1541,Women Ready-to-Wear,bottom segment,Casual Style,a066443,Business Treatment,...,2020.0,Winter Collection,Women's Fashion,Client,Outside venue,Business Engagement,1.0,1.0,0.0,1541


In [8]:
# Ensure datetime format of columns of interest 
df_merged["transaction_date"] = pd.to_datetime(df_merged["transaction_date"])
df_merged["action_start_date"] = pd.to_datetime(df_merged["action_start_date"])
df_merged["action_end_date"] = pd.to_datetime(df_merged["action_end_date"])

In [9]:
# First transaction AFTER the action begins
df_post_action = df_merged[df_merged["transaction_date"] > df_merged["action_start_date"]]

# Get the first transaction after the action start date for each client and action_id
first_transaction = df_post_action.loc[
    df_post_action.groupby(["action_id", "client_id"])["transaction_date"].idxmin(),
    ["action_id", "client_id", "action_label", "action_universe", "action_start_date", "action_end_date", "transaction_date", "total_revenue", "product_category"]
].rename(columns={"transaction_date": "first_transaction_after_date"})

# Last transaction BEFORE the action starts
df_pre_action = df_merged[df_merged["transaction_date"] < df_merged["action_start_date"]]

last_transaction = df_pre_action.loc[
    df_pre_action.groupby(["action_id", "client_id"])["transaction_date"].idxmin(),
    ["action_id", "client_id", "action_label", "action_universe", "action_start_date", "action_end_date", "transaction_date", "total_revenue", "product_category"]
].rename(columns={"transaction_date": "last_transaction_before_date"})

In [10]:
first_transaction["days_after_action"] = (
    first_transaction["first_transaction_after_date"] - first_transaction["action_start_date"]
).dt.days.astype(float)

first_transaction

,action_id,client_id,action_label,action_universe,action_start_date,action_end_date,first_transaction_after_date,total_revenue,product_category,days_after_action
32336,a000858,c11793460,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-15,1582,Women Ready-to-Wear,2.0
40377,a000858,c13965143,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-17,565,House,4.0
29294,a000858,c15700954,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-14,1384,Men Ready-to-Wear,1.0
41573,a000858,c21270108,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-24,4756,Women Ready-to-Wear,11.0
37968,a000858,c31600903,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-17,1308,Woman Shoes,4.0
...,...,...,...,...,...,...,...,...,...,...
3347,a999183,c14900851,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2021-05-06,1410,Women Ready-to-Wear,3.0
63293,a999183,c33421439,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2021-05-06,22175,Women Accessory,3.0
66009,a999183,c50315387,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2021-11-03,55522,Women Ready-to-Wear,184.0
21534,a999183,c54470231,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2021-05-06,3490,Women Ready-to-Wear,3.0


In [11]:
last_transaction["days_before_action"] = (
    last_transaction["action_start_date"] - last_transaction["last_transaction_before_date"]
).dt.days.astype(float)

last_transaction

,action_id,client_id,action_label,action_universe,action_start_date,action_end_date,last_transaction_before_date,total_revenue,product_category,days_before_action
28282,a000858,c02866010,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2020-01-30,1704,Men Shoes,745.0
30666,a000858,c02903251,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2020-02-05,1165,Women Accessory,739.0
48009,a000858,c07435416,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2020-05-02,1926,Woman Shoes,652.0
32518,a000858,c11793460,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2020-01-12,1014,Men Ready-to-Wear,763.0
77733,a000858,c13965143,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2018-11-10,3379,Women Ready-to-Wear,1191.0
...,...,...,...,...,...,...,...,...,...,...
18647,a987560,c89139006,Social Gathering,Timepieces and Jewelry,2022-03-09,2022-03-09,2020-07-07,2791,Women Small Leather Goods,610.0
57991,a987560,c89648202,Social Gathering,Timepieces and Jewelry,2022-03-09,2022-03-09,2020-01-23,933,Women Ready-to-Wear,776.0
78322,a999183,c14900851,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2019-01-21,2421,Jewelry,833.0
3210,a999183,c33421439,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2020-10-01,5451,Women Bags,214.0


In [12]:
# Group by action_id and calculate the required aggregations
summary = first_transaction.groupby('action_id').agg(
    unique_clients=('client_id', 'nunique'),
    total_transaction=('client_id', 'count'),
    total_revenue=('total_revenue', 'sum'),
    average_revenue_per_transaction = ('total_revenue', 'mean'),
    average_days_after_action=('days_after_action', 'mean')
).reset_index()

summary

,action_id,unique_clients,total_transaction,total_revenue,average_revenue_per_transaction,average_days_after_action
0,a000858,20,20,85710,4285.500000,5.900000
1,a005065,9,9,17798,1977.555556,96.222222
2,a009327,1,1,4385,4385.000000,8.000000
3,a021005,3,3,5287,1762.333333,9.333333
4,a021166,1,1,2364,2364.000000,11.000000
...,...,...,...,...,...,...
332,a977139,3,3,18197,6065.666667,33.666667
333,a980333,1,1,9862,9862.000000,543.000000
334,a981144,1,1,1932,1932.000000,352.000000
335,a982391,2,2,4090,2045.000000,54.500000


In [13]:
# Group by action_id and aggregate the required columns
grouped_df = df_actions.groupby('action_id').agg(
    action_label=('action_label', 'first'),
    action_universe=('action_universe', 'first'),
    total_client_is_invited=('client_is_invited', 'sum')
).reset_index()

# Display the grouped dataframe
grouped_df

,action_id,action_label,action_universe,total_client_is_invited
0,a000858,Trunk Show Event,Women's Fashion,76
1,a004211,Social Gathering,Women's Fashion,1
2,a005065,Exclusive Offer,Women's Fashion,10
3,a005924,Exclusive Offer,Women's Fashion,1
4,a006094,Outdoor Event,Infant and Toddler,3
...,...,...,...,...
559,a987560,Social Gathering,Timepieces and Jewelry,15
560,a988444,Exclusive Offer,Women's Fashion,1
561,a993574,Lunar New Year Celebration,Women's Fashion,1
562,a996597,Exclusive Offer,Women's Fashion,2


In [14]:
# Merge summary and grouped_df on action_id
merged_summary = summary.merge(grouped_df, on='action_id', how='left')

# Display the merged dataframe
merged_summary

,action_id,unique_clients,total_transaction,total_revenue,average_revenue_per_transaction,average_days_after_action,action_label,action_universe,total_client_is_invited
0,a000858,20,20,85710,4285.500000,5.900000,Trunk Show Event,Women's Fashion,76
1,a005065,9,9,17798,1977.555556,96.222222,Exclusive Offer,Women's Fashion,10
2,a009327,1,1,4385,4385.000000,8.000000,Trunk Show Event,Women's Fashion,1
3,a021005,3,3,5287,1762.333333,9.333333,Social Gathering,Women's Fashion,3
4,a021166,1,1,2364,2364.000000,11.000000,Holiday Celebration,Women's Fashion,3
...,...,...,...,...,...,...,...,...,...
332,a977139,3,3,18197,6065.666667,33.666667,Trunk Show Event,Women's Fashion,3
333,a980333,1,1,9862,9862.000000,543.000000,Exclusive Offer,Women's Fashion,1
334,a981144,1,1,1932,1932.000000,352.000000,Exclusive Offer,Women's Fashion,1
335,a982391,2,2,4090,2045.000000,54.500000,Exclusive Offer,Women's Fashion,2


In [15]:
# Group by action_label and action_universe and calculate the required aggregations
final_summary = merged_summary.groupby(['action_label', 'action_universe']).agg(
    distinct_actions_count=('action_id', 'nunique'),
    total_client_is_invited_sum=('total_client_is_invited', 'sum'),
    total_client_is_invited_avg=('total_client_is_invited', 'mean'),
    average_revenue_per_transaction_avg=('average_revenue_per_transaction', 'mean'),
    total_revenue_sum =('total_revenue', 'sum'),
    total_transaction_sum = ('total_transaction', 'sum'),
    average_days_after_action_avg=('average_days_after_action', 'mean')
).reset_index()

# Display the final summary
final_summary["conversion_rate"] = final_summary["total_transaction_sum"] / final_summary["total_client_is_invited_sum"]
final_summary["transaction_post_event"] = final_summary["total_transaction_sum"] / final_summary["distinct_actions_count"]

final_summary

,action_label,action_universe,distinct_actions_count,total_client_is_invited_sum,total_client_is_invited_avg,average_revenue_per_transaction_avg,total_revenue_sum,total_transaction_sum,average_days_after_action_avg,conversion_rate,transaction_post_event
0,Back-to-School Promotion,Women's Fashion,2,15,7.500000,1965.722222,27103,10,45.666667,0.666667,5.000000
1,Business Engagement,Men's Fashion,11,16,1.454545,2833.727273,63821,15,84.030303,0.937500,1.363636
2,Business Engagement,Women's Fashion,14,63,4.500000,5272.616824,294192,54,60.616104,0.857143,3.857143
3,Exclusive Client Days,Men's Fashion,1,3,3.000000,5801.500000,11603,2,17.000000,0.666667,2.000000
4,Exclusive Client Days,Women's Fashion,2,10,5.000000,3812.833333,50415,10,163.666667,1.000000,5.000000
5,Exclusive Offer,Infant and Toddler,7,21,3.000000,2412.547619,39610,17,91.485714,0.809524,2.428571
6,Exclusive Offer,Men's Fashion,18,88,4.888889,2900.818254,207099,76,168.741677,0.863636,4.222222
7,Exclusive Offer,Timepieces and Jewelry,9,43,4.777778,4643.585185,115109,33,94.309877,0.767442,3.666667
8,Exclusive Offer,Women's Fashion,138,1928,13.971014,3675.259018,3856787,1057,100.747678,0.548237,7.659420
9,Exclusive Preview,Women's Fashion,1,11,11.000000,5469.100000,54691,10,29.600000,0.909091,10.000000


In [16]:
import plotly.express as px

# Create a new column for combined labels
final_summary['label'] = final_summary['action_label'] + ' (' + final_summary['action_universe'] + ')'

# Create the scatter plot
fig = px.scatter(
    final_summary,
    x='average_revenue_per_transaction_avg',
    y='average_days_after_action_avg',
    color='action_label',
    hover_data={'label': True},
    labels={
        'average_revenue_per_transaction_avg': 'Average Revenue per Transaction',
        'average_days_after_action_avg': 'Average Days After Action'
    },
    title='Average Revenue per Transaction vs. Average Days After Action'
)

# Invert the y-axis
fig.update_yaxes(autorange='reversed', range=[75, 0])

# Limit the x-axis
fig.update_xaxes(range=[0, 6100])

# Remove the legend
fig.update_layout(showlegend=False)

# Show the plot
fig.show()

/Users/ghalichraibi/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


Estimating the 30 Days Uplift 

In [17]:
import pandas as pd

# Filter first_transaction for days_after_action < 30
filtered_first_transaction = first_transaction[first_transaction['days_after_action'] < 30]

# Filter last_transaction for days_before_action < 30
filtered_last_transaction = last_transaction[last_transaction['days_before_action'] < 30]

In [18]:
filtered_first_transaction

,action_id,client_id,action_label,action_universe,action_start_date,action_end_date,first_transaction_after_date,total_revenue,product_category,days_after_action
32336,a000858,c11793460,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-15,1582,Women Ready-to-Wear,2.0
40377,a000858,c13965143,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-17,565,House,4.0
29294,a000858,c15700954,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-14,1384,Men Ready-to-Wear,1.0
41573,a000858,c21270108,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-24,4756,Women Ready-to-Wear,11.0
37968,a000858,c31600903,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-17,1308,Woman Shoes,4.0
...,...,...,...,...,...,...,...,...,...,...
22037,a973668,c20659258,Exclusive Offer,Women's Fashion,2020-05-07,2020-06-01,2020-05-16,6374,Women Ready-to-Wear,9.0
64285,a977139,c42214090,Trunk Show Event,Women's Fashion,2021-04-12,2021-04-13,2021-04-24,5998,Women Ready-to-Wear,12.0
3347,a999183,c14900851,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2021-05-06,1410,Women Ready-to-Wear,3.0
63293,a999183,c33421439,Fur Collection Rotation,Women's Fashion,2021-05-03,2021-05-04,2021-05-06,22175,Women Accessory,3.0


In [19]:
filtered_last_transaction

,action_id,client_id,action_label,action_universe,action_start_date,action_end_date,last_transaction_before_date,total_revenue,product_category,days_before_action
2394,a000858,c25518736,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-01-19,9402,Women Bags,25.0
71080,a000858,c31029087,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-10,11017,Women Bags,3.0
25611,a000858,c40526451,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-10,1606,Women Accessory,3.0
25140,a000858,c45082951,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-11,738,Women Accessory,2.0
68290,a000858,c47811134,Trunk Show Event,Women's Fashion,2022-02-13,2022-02-15,2022-02-12,5135,Women Bags,1.0
...,...,...,...,...,...,...,...,...,...,...
56005,a970430,c92777131,Exclusive Offer,Women's Fashion,2021-07-15,2021-07-17,2021-07-05,5272,Men Bags,10.0
55952,a970430,c92926609,Exclusive Offer,Women's Fashion,2021-07-15,2021-07-17,2021-06-28,5373,Women Bags,17.0
23912,a970430,c95044835,Exclusive Offer,Women's Fashion,2021-07-15,2021-07-17,2021-06-29,1578,Women Accessory,16.0
22205,a973668,c20659258,Exclusive Offer,Women's Fashion,2020-05-07,2020-06-01,2020-05-04,760,Women Accessory,3.0


In [20]:
# Group by action_id and calculate the required aggregations for filtered_first_transaction
first_transaction_summary = filtered_first_transaction.groupby('action_id').agg(
    action_label=('action_label', 'first'),
    action_universe=('action_universe', 'first'),
    num_transactions=('client_id', 'count'),
    average_revenue_per_transaction=('total_revenue', 'mean'),
    total_revenue=('total_revenue', 'sum')
).reset_index()

# Group by action_id and calculate the required aggregations for filtered_last_transaction
last_transaction_summary = filtered_last_transaction.groupby('action_id').agg(
    action_label=('action_label', 'first'),
    action_universe=('action_universe', 'first'),
    num_transactions=('client_id', 'count'),
    average_revenue_per_transaction=('total_revenue', 'mean'),
    total_revenue=('total_revenue', 'sum')
).reset_index()

In [21]:
first_transaction_summary

,action_id,action_label,action_universe,num_transactions,average_revenue_per_transaction,total_revenue
0,a000858,Trunk Show Event,Women's Fashion,20,4285.500000,85710
1,a005065,Exclusive Offer,Women's Fashion,4,2527.000000,10108
2,a009327,Trunk Show Event,Women's Fashion,1,4385.000000,4385
3,a021005,Social Gathering,Women's Fashion,3,1762.333333,5287
4,a021166,Holiday Celebration,Women's Fashion,1,2364.000000,2364
...,...,...,...,...,...,...
226,a970430,Exclusive Offer,Women's Fashion,50,3120.580000,156029
227,a971819,Exclusive Offer,Women's Fashion,4,3655.250000,14621
228,a973668,Exclusive Offer,Women's Fashion,1,6374.000000,6374
229,a977139,Trunk Show Event,Women's Fashion,1,5998.000000,5998


In [22]:
last_transaction_summary

,action_id,action_label,action_universe,num_transactions,average_revenue_per_transaction,total_revenue
0,a000858,Trunk Show Event,Women's Fashion,13,4866.846154,63269
1,a021005,Social Gathering,Women's Fashion,1,684.000000,684
2,a025518,Trunk Show Event,Women's Fashion,1,2116.000000,2116
3,a027312,Exclusive Offer,Women's Fashion,2,4000.000000,8000
4,a065705,Exclusive Offer,Women's Fashion,1,7447.000000,7447
...,...,...,...,...,...,...
94,a933979,Social Gathering,Men's Fashion,1,9199.000000,9199
95,a951028,Exclusive Offer,Women's Fashion,1,2082.000000,2082
96,a970430,Exclusive Offer,Women's Fashion,18,4743.555556,85384
97,a973668,Exclusive Offer,Women's Fashion,1,760.000000,760


In [23]:
# Merge first_transaction_summary with last_transaction_summary on action_id using a left join
merged_summary = first_transaction_summary.merge(
    last_transaction_summary,
    on='action_id',
    how='left',
    suffixes=('_first', '_last')
)

# Drop the action_label_last and action_universe_last columns
merged_summary.drop(columns=['action_label_last', 'action_universe_last'], inplace=True)

# Replace NaN values with 0s
merged_summary.fillna(0, inplace=True)

# Create the percentage increase column
merged_summary['percentage_increase_revenue'] = (
    (merged_summary['total_revenue_first'] - merged_summary['total_revenue_last']) / merged_summary['total_revenue_last']
) * 100

# Create the difference in number of transactions column
merged_summary['difference_num_transactions'] = (
    merged_summary['num_transactions_first'] - merged_summary['num_transactions_last']
)

merged_summary

,action_id,action_label_first,action_universe_first,num_transactions_first,average_revenue_per_transaction_first,total_revenue_first,num_transactions_last,average_revenue_per_transaction_last,total_revenue_last,percentage_increase_revenue,difference_num_transactions
0,a000858,Trunk Show Event,Women's Fashion,20,4285.500000,85710,13.0,4866.846154,63269.0,35.469187,7.0
1,a005065,Exclusive Offer,Women's Fashion,4,2527.000000,10108,0.0,0.000000,0.0,inf,4.0
2,a009327,Trunk Show Event,Women's Fashion,1,4385.000000,4385,0.0,0.000000,0.0,inf,1.0
3,a021005,Social Gathering,Women's Fashion,3,1762.333333,5287,1.0,684.000000,684.0,672.953216,2.0
4,a021166,Holiday Celebration,Women's Fashion,1,2364.000000,2364,0.0,0.000000,0.0,inf,1.0
...,...,...,...,...,...,...,...,...,...,...,...
226,a970430,Exclusive Offer,Women's Fashion,50,3120.580000,156029,18.0,4743.555556,85384.0,82.737984,32.0
227,a971819,Exclusive Offer,Women's Fashion,4,3655.250000,14621,0.0,0.000000,0.0,inf,4.0
228,a973668,Exclusive Offer,Women's Fashion,1,6374.000000,6374,1.0,760.000000,760.0,738.684211,0.0
229,a977139,Trunk Show Event,Women's Fashion,1,5998.000000,5998,0.0,0.000000,0.0,inf,1.0


In [24]:
import numpy as np

# Create a column that returns 1 if percentage_increase_revenue is inf and 0 otherwise
merged_summary['is_new'] = np.where(np.isinf(merged_summary['percentage_increase_revenue']), 1, 0)

# Separate the DataFrame into two based on the is_inf column
df_new = merged_summary[merged_summary['is_new'] == 1]
df_not = merged_summary[merged_summary['is_new'] == 0]

In [25]:
df_new

,action_id,action_label_first,action_universe_first,num_transactions_first,average_revenue_per_transaction_first,total_revenue_first,num_transactions_last,average_revenue_per_transaction_last,total_revenue_last,percentage_increase_revenue,difference_num_transactions,is_new
1,a005065,Exclusive Offer,Women's Fashion,4,2527.000,10108,0.0,0.0,0.0,inf,4.0,1
2,a009327,Trunk Show Event,Women's Fashion,1,4385.000,4385,0.0,0.0,0.0,inf,1.0,1
4,a021166,Holiday Celebration,Women's Fashion,1,2364.000,2364,0.0,0.0,0.0,inf,1.0,1
5,a022074,Social Gathering,Men's Fashion,1,5020.000,5020,0.0,0.0,0.0,inf,1.0,1
6,a025108,Exclusive Offer,Timepieces and Jewelry,1,5871.000,5871,0.0,0.0,0.0,inf,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
224,a948575,Exclusive Offer,Women's Fashion,1,1168.000,1168,0.0,0.0,0.0,inf,1.0,1
225,a968576,Trunk Show Event,Women's Fashion,8,7712.125,61697,0.0,0.0,0.0,inf,8.0,1
227,a971819,Exclusive Offer,Women's Fashion,4,3655.250,14621,0.0,0.0,0.0,inf,4.0,1
229,a977139,Trunk Show Event,Women's Fashion,1,5998.000,5998,0.0,0.0,0.0,inf,1.0,1


In [26]:
# Group by action_label_first and action_universe_first and calculate the required aggregations
grouped_new_summary = df_new.groupby(['action_label_first', 'action_universe_first']).agg(
    distinct_action_id_count=('action_id', 'nunique'),
    average_total_revenue_first=('total_revenue_first', 'mean')
).reset_index()

# Display the grouped summary
grouped_new_summary

,action_label_first,action_universe_first,distinct_action_id_count,average_total_revenue_first
0,Back-to-School Promotion,Women's Fashion,2,3709.000000
1,Business Engagement,Men's Fashion,6,9230.666667
2,Business Engagement,Women's Fashion,8,4718.250000
3,Exclusive Client Days,Men's Fashion,1,7449.000000
4,Exclusive Client Days,Women's Fashion,2,1787.500000
5,Exclusive Offer,Infant and Toddler,3,6301.000000
6,Exclusive Offer,Men's Fashion,8,3468.875000
7,Exclusive Offer,Timepieces and Jewelry,4,7089.750000
8,Exclusive Offer,Women's Fashion,51,11472.745098
9,Fur Collection Rotation,Women's Fashion,1,27075.000000


In [27]:
df_not

,action_id,action_label_first,action_universe_first,num_transactions_first,average_revenue_per_transaction_first,total_revenue_first,num_transactions_last,average_revenue_per_transaction_last,total_revenue_last,percentage_increase_revenue,difference_num_transactions,is_new
0,a000858,Trunk Show Event,Women's Fashion,20,4285.500000,85710,13.0,4866.846154,63269.0,35.469187,7.0,0
3,a021005,Social Gathering,Women's Fashion,3,1762.333333,5287,1.0,684.000000,684.0,672.953216,2.0,0
7,a025518,Trunk Show Event,Women's Fashion,13,4404.230769,57255,1.0,2116.000000,2116.0,2605.812854,12.0,0
8,a027312,Exclusive Offer,Women's Fashion,9,4724.333333,42519,2.0,4000.000000,8000.0,431.487500,7.0,0
14,a065705,Exclusive Offer,Women's Fashion,1,1958.000000,1958,1.0,7447.000000,7447.0,-73.707533,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
212,a881751,Exclusive Offer,Women's Fashion,10,1565.900000,15659,4.0,1140.500000,4562.0,243.248575,6.0,0
215,a890054,Exclusive Offer,Women's Fashion,9,4342.888889,39086,5.0,2635.200000,13176.0,196.645416,4.0,0
221,a932079,Social Gathering,Women's Fashion,26,1753.423077,45589,13.0,5906.384615,76783.0,-40.626180,13.0,0
226,a970430,Exclusive Offer,Women's Fashion,50,3120.580000,156029,18.0,4743.555556,85384.0,82.737984,32.0,0


In [28]:
# Group by action_label_first and action_universe_first and calculate the required averages
grouped_summary = df_not.groupby(['action_label_first', 'action_universe_first']).agg(
    average_difference_num_transactions=('difference_num_transactions', 'mean'),
    average_percentage_increase_revenue=('percentage_increase_revenue', 'mean')
).reset_index()

# Display the grouped summary
grouped_summary

,action_label_first,action_universe_first,average_difference_num_transactions,average_percentage_increase_revenue
0,Business Engagement,Women's Fashion,5.000000,448.307658
1,Exclusive Offer,Infant and Toddler,2.000000,156.881782
2,Exclusive Offer,Men's Fashion,7.000000,1870.453090
3,Exclusive Offer,Timepieces and Jewelry,10.000000,140.433990
4,Exclusive Offer,Women's Fashion,6.868421,1710.481895
5,Exclusive Preview,Women's Fashion,4.000000,161.485390
6,Holiday Celebration,Women's Fashion,108.600000,2112.367291
7,Lunar New Year Celebration,Women's Fashion,8.500000,3048.056070
8,Luxurious Fur Showcase,Women's Fashion,0.000000,738.684211
9,Miscellaneous Social Gathering,Women's Fashion,12.000000,3575.733292
